In [1]:
import xarray as xr
import intake
import numpy as np
from cmip6_preprocessing.preprocessing import combined_preprocessing
from cmip6_preprocessing.postprocessing import match_metrics, combine_datasets, merge_variables
from dask.diagnostics import ProgressBar

In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

In [4]:
variables = ['fgco2','hfds']
z_kwargs = {'consolidated': True, 'use_cftime': True}
kwargs = dict(zarr_kwargs=z_kwargs, preprocess=combined_preprocessing, aggregate=False)

query = dict(experiment_id=['piControl'], 
             #table_id=['Omon'], 
             variable_id=variables,
             #grid_label=['gn']
            )
cat = col.search(**query)
dd = cat.to_dataset_dict(**kwargs)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [5]:
def pick_first_member(ds_list, **kwargs):
    return ds_list[0]

ddnow = merge_variables(dd)
ddnow = combine_datasets(
    ddnow,
    pick_first_member,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)
list(ddnow.keys())

/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.MOHC.UKESM1-0-LL.piControl.r1i1p1f2.Omon.gn.none failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.NASA-GISS.GISS-E2-1-G.piControl.r102i1p1f1.Omon.gn.none failed to combine with :indexes along dimension 'y' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.IPSL.IPSL-CM6A-LR.piControl.r1i1p1f1.Omon.gn.none failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.8/site-packages/cmip6_preprocessing/postprocessin

['CMCC-CM2-SR5.gn.piControl.Omon',
 'FGOALS-g3.gn.piControl.Omon',
 'AWI-ESM-1-1-LR.gn.piControl.Omon',
 'IPSL-CM6A-LR.gn.piControl.Oyr',
 'CESM2.gn.piControl.Omon',
 'CNRM-CM6-1-HR.gn.piControl.Omon',
 'SAM0-UNICON.gn.piControl.Omon',
 'GFDL-CM4.gr.piControl.Omon',
 'MRI-ESM2-0.gn.piControl.Omon',
 'ACCESS-ESM1-5.gn.piControl.Omon',
 'GISS-E2-1-H.gn.piControl.Omon',
 'MPI-ESM1-2-LR.gn.piControl.Omon',
 'E3SM-1-1.gr.piControl.Omon',
 'CESM2-WACCM.gn.piControl.Oyr',
 'NorESM1-F.gn.piControl.Omon',
 'CESM2-FV2.gr.piControl.Omon',
 'FIO-ESM-2-0.gn.piControl.Omon',
 'CIESM.gn.piControl.Omon',
 'MCM-UA-1-0.gn.piControl.Omon',
 'CanESM5.gn.piControl.Omon',
 'AWI-CM-1-1-MR.gn.piControl.Omon',
 'NorCPM1.gn.piControl.Omon',
 'IPSL-CM6A-LR.gn.piControl.Omon',
 'GISS-E2-1-G.gn.piControl.Omon',
 'ACCESS-ESM1-5.gn.piControl.Oyr',
 'MIROC-ES2L.gr1.piControl.Omon',
 'GFDL-ESM4.gr.piControl.Omon',
 'EC-Earth3-Veg-LR.gn.piControl.Omon',
 'MPI-ESM1-2-HR.gn.piControl.Oyr',
 'MPI-ESM1-2-HR.gn.piControl.Om

In [7]:
ddsub = {}
for d in ddnow.keys():
    vs = ddnow[d].data_vars
    if ('hfds' in vs) & ('fgco2' in vs):# & ('intpp' in vs):
        ddsub[d] = ddnow[d]

In [11]:
cat_metric = col.search(
    variable_id=['areacello'], 
    table_id=['Ofx'], 
    experiment_id=['piControl'])
dd_metric = cat_metric.to_dataset_dict(**kwargs)

dddone = match_metrics(ddsub, dd_metric, ['areacello'])


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [15]:
ddint = {}
for key in dddone.keys():
    print(key)
    ds = dddone[key]
    ddint[key] = ds.weighted(ds['areacello'].fillna(0)).sum(['x', 'y']).resample(time='1AS').mean()
    with ProgressBar():
        ddint[key].to_netcdf('../data/processed/global_integrals/'+key+'.nc')

CESM2.gn.piControl.Omon
[########################################] | 100% Completed |  1min 48.3s
MRI-ESM2-0.gn.piControl.Omon
[########################################] | 100% Completed | 21.6s
ACCESS-ESM1-5.gn.piControl.Omon
[########################################] | 100% Completed |  1min  8.3s
MPI-ESM1-2-LR.gn.piControl.Omon
[########################################] | 100% Completed |  6.3s
NorESM1-F.gn.piControl.Omon
[########################################] | 100% Completed | 18.9s
CESM2-FV2.gr.piControl.Omon
[########################################] | 100% Completed | 27.9s
CanESM5.gn.piControl.Omon
[########################################] | 100% Completed |  1min 17.6s
NorCPM1.gn.piControl.Omon
[########################################] | 100% Completed | 44.9s
IPSL-CM6A-LR.gn.piControl.Omon
[########################################] | 100% Completed | 18.1s
GFDL-ESM4.gr.piControl.Omon
[########################################] | 100% Completed | 24.9s
CESM2-WACCM-FV2.gr